# Data prep for estimating models

Sam Maurer, August 2017 | Python 3.6

Original version June 2017  
Updated Aug 2017 (v02) to fix int/float issues

This notebook generates the data tables that are used the model estimation demos. For more about the California Household Travel Survey source data, you can refer to the "CHTS-exploration" notebook.

In [1]:
import numpy as np
import pandas as pd
import zipfile

## Load raw CHTS tables

This requires the file named caltrans_full_survey.zip. You can download it by following the instructions in the "data" directory.

In [2]:
z = zipfile.ZipFile('../data/caltrans_full_survey.zip')

In [3]:
households = pd.read_csv(z.open('caltrans_full_survey/survey_households.csv'), low_memory=False)
len(households)

42426

In [4]:
persons = pd.read_csv(z.open('caltrans_full_survey/survey_person.csv'), low_memory=False)
len(persons)

109113

In [5]:
places = pd.read_csv(z.open('caltrans_full_survey/survey_place.csv'), low_memory=False)
len(places)

460524

In [6]:
activities = pd.read_csv(z.open('caltrans_full_survey/survey_activity.csv'), low_memory=False)
len(activities)

604711

## Clean up the places table and generate tract identifiers

In [7]:
# Discard places with missing identifiers, and convert ID components to ints. 
# (Some identifiers are stored as floats in the source table, but the 
# "CHTS-exploration" notebook confirms that the decimal vlaues don't encode anything.)

places.dropna(subset=['state_id','county_id','tract_id','city'], inplace=True)

places['state_id'] = places.state_id.astype(int)
places['county_id'] = places.county_id.astype(int)
places['tract_id'] = places.tract_id.astype(int)

print(len(places))

460516


In [8]:
# Other missing values are encoded as nines; discard those as well

places.drop((places.tract_id == 999999) | 
            (places.county_id == 999) | 
            (places.state_id == 99), inplace=True)

print(len(places))

460514


In [9]:
# Clean up other data fields

# Replace null travel mode with zero and encode as int (mode seems to be a protected
# keyword, so we have to use places['mode'] rather than places.mode)

places['mode'] = places['mode'].fillna(0).astype(int)

print(places['mode'].head(3))

2    6
3    0
4    5
Name: mode, dtype: int64


In [10]:
# Define functions to move back and forth between full numerical tract ID and its components

def full_tract_id(state_id, county_id, tract_id):
    return state_id * 10**9 + county_id * 10**6 + tract_id

def state_id(full_tract_id):
    return full_tract_id // 10**9

def county_id(full_tract_id):
    _county_tract = np.fmod(full_tract_id, 10**9)
    return _county_tract // 10**6

print(full_tract_id(6, 98, 941414))
print(state_id(6098141414))
print(county_id(6098141414))

6098941414
6
98


In [11]:
# Generate full tract identifiers

places['full_tract_id'] = full_tract_id(places.state_id, places.county_id, places.tract_id)

print(places[['state_id','county_id','tract_id','city','full_tract_id']].head(3))

   state_id  county_id  tract_id     city  full_tract_id
2         6         95    252202  VALLEJO     6095252202
3         6         95    252202  VALLEJO     6095252202
4         6         95    251902  VALLEJO     6095251902


## Build a master table of census tracts

In [12]:
# Generate a master list of census tracts, keeping the city name most commonly 
# associated with each tract

tracts = places[['full_tract_id','city']].groupby('full_tract_id').\
         agg(lambda x:x.value_counts().index[0])

print(tracts.shape[0])
print(tracts.head())

9310
                     city
full_tract_id            
1015000800       ANNISTON
1101001500     MONTGOMERY
1161400100        SEVILLA
2020001000      ANCHORAGE
2020001100      ANCHORAGE


In [13]:
# Limit to the 9-county San Francisco Bay Area

tracts = tracts[(state_id(tracts.index).isin([6])) & 
                (county_id(tracts.index).\
                 isin([1, 13, 41, 55, 75, 81, 85, 95, 97]))].copy()

print(tracts.shape[0])
print(tracts.head())

1583
                   city
full_tract_id          
6001008309      TIJUANA
6001400100     BERKELEY
6001400200      OAKLAND
6001400300      OAKLAND
6001400400      OAKLAND


In [14]:
print(county_id(tracts.index).value_counts())

85    371
1     360
13    207
75    195
81    158
97     99
95     97
41     55
55     41
Name: full_tract_id, dtype: int64


## Calculate some tract-level covariates

Residential density, school/employment density

In [15]:
# Note: the `home_tract_id` in the households table is already a full 11-digit
# identifier, with the same format that we generated for the places table.
# Same with `empl_tract_id` and `school_tract_id` in the persons table.

In [16]:
# Residential density = sum of weighted household sizes by census tract of home

households['_weighted_persons_count'] = households.persons_count * households.hhwgt

home_density = households.groupby('home_tract_id')._weighted_persons_count.sum().\
        rename('home_density').to_frame()

In [17]:
# Employment density = sum of person weights by census tract of work location

work_density = persons.groupby('empl_tract_id').perwgt.sum().\
        rename('work_density').to_frame()

In [18]:
# School density = sum of person weights by census tract of school location

school_density = persons.groupby('school_tract_id').perwgt.sum().\
        rename('school_density').to_frame()

In [19]:
# Merge these into the census tracts table, only keeping Bay Area tracts

tracts = pd.merge(tracts, home_density, how='left', left_index=True, right_index=True)
tracts = pd.merge(tracts, work_density, how='left', left_index=True, right_index=True)
tracts = pd.merge(tracts, school_density, how='left', left_index=True, right_index=True)
tracts = tracts.fillna(0)  # fill missing values with zero

print(tracts.head())

                   city  home_density  work_density  school_density
full_tract_id                                                      
6001008309      TIJUANA      0.000000      0.000000        0.000000
6001400100     BERKELEY     13.437961     13.130867       13.511570
6001400200      OAKLAND     11.089638      4.248928        0.894794
6001400300      OAKLAND     28.878399      7.671554        0.000000
6001400400      OAKLAND     16.884910      4.063805        8.150402


## Generate a table of trips

For now, this is a table of places visited for non-school, non-work activities

In [20]:
# - trip destinations are in `places.full_tract_id` (sometimes missing)
# - trip purposes are in `activities.purpose`, and we want 23 thru 38
# - places and acitivities are linked by `sampno`, `perno`, `plano`, and there 
#   can be multiple activities per place

In [21]:
# Function to generate a single unique ID for places

def place_id(sampno, perno, plano):
    return sampno * 10**4 + perno * 10**2 + plano

print(place_id(1041766, 3, 12))

10417660312


In [22]:
# Add place_id to places table and activities table

places['place_id'] = place_id(places.sampno, places.perno, places.plano)
activities['place_id'] = place_id(activities.sampno, activities.perno, activities.plano)

In [23]:
# Get list of places that have a secondary activity

_secondary_activity_places = activities.loc[activities.purpose.isin(range(23, 38+1)),
                                            'place_id'].drop_duplicates()

In [24]:
# Generate a table of those places with some covariates

trips = places.loc[places.place_id.isin(_secondary_activity_places) &
                   places.full_tract_id.notnull(),
                  ['place_id', 'full_tract_id', 'mode', 
                   'trip_distance_miles']].set_index('place_id')

print(trips.shape[0])
print(trips.head())

147004
             full_tract_id  mode  trip_distance_miles
place_id                                             
10319850202     6095251902     5             5.125960
10320360102     6073017051     5             3.619056
10320360104     6073009304     5            19.351620
10320360105     6073008511     5             6.451126
10320360202     6073020211     6            10.466616


In [25]:
# Limit to destinations in the 9-county San Francisco Bay Area

trips = trips[(state_id(trips.full_tract_id).isin([6])) & 
              (county_id(trips.full_tract_id).\
               isin([1, 13, 41, 55, 75, 81, 85, 95, 97]))].copy()

print(trips.shape[0])
print(trips.head())

36764
             full_tract_id  mode  trip_distance_miles
place_id                                             
10319850202     6095251902     5             5.125960
10335860102     6085511915     6           156.370628
10335860103     6085512027     6             1.615535
10335860104     6085512027     6             0.375708
10335860105     6085511915     6             0.894730


## Save estimaton data to disk

In [26]:
tracts.to_csv('../data/tracts_v02.csv')

In [27]:
trips.to_csv('../data/trips_v02.csv')

In [ ]:
# TO DO
# - for a mode choice model, could probably generate average travel times between
#   tracts just from the observed data